In [ ]:
import pandas as pd

In [ ]:
import datetime
# datetime_str = '2022-01-01 00:00:00'

def get_unix_epoch_time_ms(datetime_str):
    datetime_obj = datetime.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
    unix_epoch_time_ms = int(datetime_obj.timestamp() * 1000)
    return unix_epoch_time_ms

In [ ]:
# target = datetime.datetime.now()
target = pd.to_datetime('2022-07-19 10:49:00') 
delta = datetime.timedelta(days=120)
stamps = []
for i in range(1, 289):
    dt = target - datetime.timedelta(minutes=i*1000)
    if dt < target - delta:
        break
    stamps.append(dt.strftime('%Y-%m-%d %H:%M:%S'))

stamps = stamps[::-1]
pairs = [[stamps[i], stamps[i+1]] for i in range(0, len(stamps), 2)]


In [ ]:
from binance.um_futures import UMFutures

um_futures_client = UMFutures()

# :parameter symbol: string; the trading symbol.
# :parameter interval: string; the interval of kline, e.g 1m, 5m, 1h, 1d, etc. (see more in https://binance-docs.github.io/apidocs/futures/en/#public-endpoints-info)
# :parameter limit: optional int; limit the results. Default 500, max 1000.
# :parameter startTime: optional int
# :parameter endTime: optional int

# resp = um_futures_client.klines(symbol="BTCUSDT", interval="1m", limit=1000)
# std_length = len(resp[0])
# for i in resp:
#     assert len(i) == std_length

resp_data = []
for pairs in pairs:
    start = get_unix_epoch_time_ms(pairs[0])
    end = get_unix_epoch_time_ms(pairs[1])
    resp = um_futures_client.klines(symbol="BTCUSDT", interval="1m", startTime=start, endTime=end)
    resp_data.extend(resp)  


In [ ]:
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.

In [ ]:

# Create an empty dataframe
df = pd.DataFrame(columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'num_of_trades', 'taker_buy_base', 'taker_buy_quote'])

# Loop through each item in resp and append to the dataframe
for item in resp_data:
    timestamp, oopen, high, low, close, volume, close_time, quote_asset_volume, num_of_trades, taker_buy_base, taker_buy_quote, ignore = item
      # create a dictionary of the values
    data = {'timestamp': timestamp, 'open': oopen, 'high': high, 'low': low, 'close': close, 'volume': volume, 'close_time': close_time, 'quote_asset_volume': quote_asset_volume, 'num_of_trades': num_of_trades, 'taker_buy_base': taker_buy_base, 'taker_buy_quote': taker_buy_quote}
    # append the dictionary as a row to the dataframe
    df = pd.concat([df, pd.DataFrame(data, index=[timestamp])])

# Set timestamp as the index
# convert timestamp column to datetime and set it as index
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df

In [ ]:
# # calculate EMA 200 and EMA 50
# df['EMA_200'] = df['close'].rolling(window=200).mean()
# df['EMA_50'] = df['close'].rolling(window=50).mean()
# # Calculate moving average and standard deviation
# window_size = 20
# df['mid_band'] = df['close'].rolling(window=window_size).mean()
# df['std_dev'] = df['close'].rolling(window=window_size).std()
# # Calculate upper and lower Bollinger Bands
# num_std = 2
# df['upper_band'] = df['mid_band'] + (df['std_dev'] * num_std)
# df['lower_band'] = df['mid_band'] - (df['std_dev'] * num_std)
# df.dropna()
# df = df[200:]
# df

In [ ]:
df.to_csv('data/BTCUSDT_1m_3m_back_from_2022-07-19.csv')